In [3]:
import sys
sys.path.append('../utilities/')
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import torch
from sentence_transformers import SentenceTransformer
from joblib import dump
from openai import OpenAI
from tqdm import tqdm
from mmd import MMD
import re
from sklearn.feature_extraction.text import CountVectorizer

/Users/pranitgunjal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=API_KEY)

In [3]:
instruction = (
    "You are a data generator tasked with creating realistic DOTA 2 chat messages. "
    "These chat messages should be labeled according to their sentiment: toxic or non-toxic.\n"
    "Base the style on typical video game chat messages — include informal internet language, typos, and abbreviations\n"
    "Generate exactly 10 realistic DOTA 2 chat messages, one per line.\n"
    "Each line should follow this format: the chat message in double quotes, followed by a space and then the label (0 for toxic, 1 for non-toxic).\n"
    "No extra formatting — just plain text output, one line per comment.\n"
    "Here is the format:\n"
    "\"gg dawg\" 0\n"
    "\"I hate u bitch\" 1"
)
input = (
    f"Now, generate the 10 new comments below:"
)

In [4]:
res = []
for i in range(100):
    response = client.responses.create(
        model="gpt-4o",
        instructions=instruction,
        input=input
    )
    res.append(response.output_text)

In [8]:
labels = []
sentences = []
for i in range(100):
    for word in res[i].split("\n"):
        match = re.match(r'"(.*?)"\s*(-?\d+)', word)
        if match:
            quoted = match.group(1)      
            label = match.group(2)       
            sentences.append(quoted)
            labels.append(int(label))

In [9]:
generated_df = pd.DataFrame({
    'sentences': sentences,
    'labels': labels
})

In [10]:
generated_df

,sentences,labels
0,I'm reporting u aftr game,0
1,"nice play team, wp!",1
2,wtf r u doin?,0
3,"keep it up, ur doing gr8!",1
4,lol uninstall pls,0
...,...,...
995,"let's push mid, we got this",1
996,"u can't last hit, uninstall",0
997,nice save back there,1
998,"stop throwing, wtf",0


In [12]:
generated_df.to_csv('../data/generated/dota2/control/control_gen.csv', index=False)